In [79]:
import torch

from nn_pruning.modules.binarizer import MagnitudeBinarizer, ThresholdBinarizer, TopKBinarizer

def ampere_mask_threshold(mask_scores, threshold, sigmoid, train):
    assert((mask_scores.shape[1] % 4) == 0)
        
    mask_scores_4 = mask_scores.reshape(mask_scores.shape[0], mask_scores.shape[1] // 4, 4)
    top = torch.topk(mask_scores_4, 2, dim=2, largest=True)    
    top_mask = torch.zeros_like(mask_scores_4, device=mask_scores.device)        
    top_mask = top_mask.scatter(2, top.indices, True)        
    top_mask = top_mask.reshape_as(mask_scores)
    
    if train:
        mask = ThresholdBinarizer.apply(mask_scores, threshold, sigmoid)    
        ret = torch.max(mask, top_mask)
    else:
        ret = top_mask
    
    return ret
    
    
scores = torch.randn(11,12)

mask = ampere_mask_threshold(scores, 0.9, True, False)

assert(mask.shape == scores.shape)

print(mask)


tensor([[0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0.],
        [0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1.],
        [1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1.],
        [1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1.],
        [0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.],
        [0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0.],
        [1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0.],
        [1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.]])
